## 05_finetune_transformers.ipynb — Дообучение языковых моделей на собственных данных

В этом ноутбуке реализовано дообучение (fine-tuning) современных языковых моделей (Roberta, MiniLM) на собственных данных по вакансиям hh.ru.

### Для чего нужен этот файл?

- Здесь происходит адаптация предобученных трансформеров (Roberta, MiniLM) к конкретным задачам HR-аналитики:
    - **извлечение навыков (Skill-NER)** из описания вакансий,
    - **классификация грейда (junior/middle/senior/lead)** по тексту вакансии.
- В ноутбуке описан процесс подготовки данных, запуска обучения, сохранения моделей и анализа качества.
- Результаты этого этапа лягут в основу интеллектуальных функций Telegram-бота и позволят применять ML к реальным задачам рынка труда.

**Задачи ноутбука:**
- Подготовить и загрузить датасеты для дообучения моделей.
- Запустить fine-tuning Roberta для задачи извлечения навыков.
- Запустить fine-tuning MiniLM (или Roberta) для задачи классификации грейда.
- Оценить качество моделей на отложенной выборке.
- Сохранить обученные модели для дальнейшей интеграции в проект.


In [1]:
%cd /content/drive/MyDrive/hh-hr-bot

/content/drive/MyDrive/hh-hr-bot


In [3]:
# Загружаем обучающую выборку для дообучения моделей Skill-NER (Roberta) и классификатора грейда
# ner_df — содержит пары (описание вакансии, список навыков), используется для обучения Skill-NER
# grade_df — содержит описание/название и метку класса грейда, для обучения классификатора грейда

import pandas as pd
import json

# Загружаем датасет для Skill-NER (jsonlines с парами "description", "skills_list")
with open('data/raw/ner_train.json', 'r', encoding='utf-8') as f:
    lines = [json.loads(line) for line in f]
ner_df = pd.DataFrame(lines)

# Просматриваем первые строки и размер выборки
display(ner_df.head(), ner_df.shape)

# Загружаем датасет для классификатора грейда (csv с колонками "description", "title", "grade_class")
grade_df = pd.read_csv('data/raw/grade_train.csv')
display(grade_df.head(), grade_df.shape)

,description,skills_list
0,<p>Вакансия &quot;Водитель с личным автомобиле...,[вождение автомобилей представительского класс...
1,<p><strong>О компании и команде</strong></p> <...,"[javascript, react, css, node.js, typescript, ..."
2,<p>​​​​​​Направление студенческих медицинских ...,"[ответственность, стрессоустойчивость]"
3,<div> <p><strong>edna – аккредитованная IT-ком...,"[react, html, css, redux, typescript, webpack]"
4,<p><strong>CAD Exchanger</strong> – IT-компани...,"[git, javascript, 3d моделирование, ооп, webgl..."


(1709, 2)

,description,title,grade_class
0,<p>Вакансия &quot;Водитель с личным автомобиле...,Водитель с личным автомобилем,3
1,<p><strong>О компании и команде</strong></p> <...,Middle/Senior Frontend разработчик,2
2,"<p><strong><em>Крупная, стабильно развивающаяс...",Упаковщик,0
3,<p><strong>Приглашаем Управляющего семейным ре...,Управляющий рестораном семейного концепта,1
4,<p>​​​​​​Направление студенческих медицинских ...,Медицинский работник (Российские студенческие ...,0


(2995, 3)

### Подготовка данных для обучения модели извлечения навыков (Skill-NER)

На этом этапе мы формируем обучающую выборку для задачи многоклассовой классификации (multilabel classification) навыков по тексту вакансии:

- Из всего корпуса вакансий собирается список уникальных навыков.
- Для каждой строки создаётся бинарный вектор признаков (“1”, если навык присутствует в описании; “0”, если нет).
- Таким образом, каждая обучающая пара — это текст описания и массив меток, соответствующих наличию навыков.
- Такой формат идеально подходит для дообучения современных трансформеров (Roberta, MiniLM) на задачу multilabel classification.

Этот шаг позволяет подготовить данные к обучению модели, способной автоматически определять набор требуемых навыков по любому тексту вакансии.


In [4]:
# Формируем список всех уникальных навыков по датасету
all_skills = set()
for skills in ner_df['skills_list']:
    all_skills.update(skills)
all_skills = sorted(list(all_skills))
print("Уникальных навыков:", len(all_skills))
print("Примеры навыков:", all_skills[:10])

Уникальных навыков: 2428
Примеры навыков: [');', '.net core', '.net framework', '1c', '1c erp', '1c: erp', '1c: бухгалтерия', '1c: зарплата и кадры', '1c: предприятие', '1водительское удостоверение категории b']


In [6]:
# Создаём бинарный вектор для каждого примера (1 — навык присутствует)
import numpy as np

def skills_to_vector(skills):
    return [int(skill in skills) for skill in all_skills]

ner_df['target_vector'] = ner_df['skills_list'].apply(skills_to_vector)
print("Пример target_vector:", ner_df['target_vector'].iloc[0])
# Смотрим пример: текст, список навыков, бинарный вектор
display(ner_df[['description', 'skills_list', 'target_vector']].head())

Пример target_vector: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

,description,skills_list,target_vector
0,<p>Вакансия &quot;Водитель с личным автомобиле...,[вождение автомобилей представительского класс...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,<p><strong>О компании и команде</strong></p> <...,"[javascript, react, css, node.js, typescript, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,<p>​​​​​​Направление студенческих медицинских ...,"[ответственность, стрессоустойчивость]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,<div> <p><strong>edna – аккредитованная IT-ком...,"[react, html, css, redux, typescript, webpack]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,<p><strong>CAD Exchanger</strong> – IT-компани...,"[git, javascript, 3d моделирование, ооп, webgl...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


### Подготовка данных для классификации грейда вакансии

На этом этапе мы анализируем и балансируем обучающий датасет для задачи классификации грейда (junior/middle/senior/lead):
- Смотрим распределение по классам, чтобы выявить возможный дисбаланс в данных.
- Проверяем примеры для каждого класса — это поможет понять специфику описаний вакансий и качество разметки.
- Этот анализ важен для успешного дообучения трансформера и повышения точности предсказания грейда вакансии.


In [8]:
# Показываем, сколько примеров в каждом классе (0=junior, 1=middle, 2=senior, 3=lead)
print(grade_df['grade_class'].value_counts())

grade_class
1    1315
0     774
2     688
3     218
Name: count, dtype: int64


In [9]:
# Посмотрим по 2 примера для каждого грейда
for grade in sorted(grade_df['grade_class'].unique()):
    print(f"\n==== Грейд: {grade} ====")
    print(grade_df[grade_df['grade_class'] == grade][['title', 'description']].head(2))


==== Грейд: 0 ====
                                               title  \
2                                          Упаковщик   
4  Медицинский работник (Российские студенческие ...   

                                         description  
2  <p><strong><em>Крупная, стабильно развивающаяс...  
4  <p>​​​​​​Направление студенческих медицинских ...  

==== Грейд: 1 ====
                                       title  \
3  Управляющий рестораном семейного концепта   
6                     Junior Web-разработчик   

                                         description  
3  <p><strong>Приглашаем Управляющего семейным ре...  
6  <p><strong>CAD Exchanger</strong> – IT-компани...  

==== Грейд: 2 ====
                                title  \
1  Middle/Senior Frontend разработчик   
5                Frontend-разработчик   

                                         description  
1  <p><strong>О компании и команде</strong></p> <...  
5  <div> <p><strong>edna – аккредитованная IT-ком...  

==== Г

### Формирование обучающей и тестовой выборки для классификатора грейда

- Для повышения качества классификации объединяем название и описание вакансии в единый текстовый признак.
- Разбиваем выборку на обучающую (80%) и тестовую (20%) части, сохраняя пропорции классов (stratify).
- Такой подход позволяет учесть максимум информации о вакансии и обеспечивает объективную оценку качества модели на независимых данных.


In [10]:
# Соединим title и description для лучшего контекста
grade_df['text'] = grade_df['title'].fillna('') + ' ' + grade_df['description'].fillna('')

In [11]:
from sklearn.model_selection import train_test_split

X = grade_df['text'].values
y = grade_df['grade_class'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

print("Train size:", len(X_train))
print("Test size:", len(X_test))

Train size: 2396
Test size: 599


### Baseline-модель для классификации грейда вакансии

Для быстрой проверки пригодности данных и минимальной оценки качества используем простую ML-модель: векторизацию текста TF-IDF и классификатор LogisticRegression.
- Такая модель даёт baseline-результат, с которым можно сравнивать более сложные подходы (Roberta, MiniLM).
- Метрики (accuracy, precision, recall, f1) покажут, насколько текст вакансии отражает уровень позиции.


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Векторизация текста
vectorizer = TfidfVectorizer(max_features=10000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Обучение логистической регрессии
clf = LogisticRegression(max_iter=1000, random_state=42)
clf.fit(X_train_tfidf, y_train)

# Предсказание на тестовой выборке
y_pred = clf.predict(X_test_tfidf)

# Оценка качества
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Классификационный отчёт:\n", classification_report(y_test, y_pred, digits=3))

Accuracy: 0.5742904841402338
Классификационный отчёт:
               precision    recall  f1-score   support

           0      0.738     0.490     0.589       155
           1      0.558     0.837     0.670       263
           2      0.465     0.341     0.393       138
           3      1.000     0.023     0.045        43

    accuracy                          0.574       599
   macro avg      0.690     0.423     0.424       599
weighted avg      0.615     0.574     0.540       599



#### Выводы по baseline-модели классификации грейда (TF-IDF + LogisticRegression)

- **Accuracy (доля правильных ответов):** 0.57 — базовая модель верно классифицирует около 57% вакансий по грейду.
- **Лучше всего определяется класс "middle" (recall 0.84)** — модель часто относит вакансии к нему (но precision ниже 0.56, возможен перекос).
- **Junior определяется достаточно уверенно** (precision 0.74, f1-score 0.59), но recall ниже (0.49) — то есть не все junior-вакансии ловит.
- **Senior/lead определяются хуже**:
    - Для senior (2): f1-score 0.39 — низкий recall (0.34), часть таких вакансий путается с другими классами.
    - Для lead (3): очень высокая precision (1.00), но recall только 0.02 (модель почти не предсказывает этот класс, f1-score всего 0.04).
- **Макро-усреднённый f1-score:** 0.42 — базовый ориентир для улучшения.

##### Причины и интерпретация:
- **Перекос классов:** больше всего примеров middle/junior, меньше lead — модель склонна "игнорировать" редкие классы.
- **TF-IDF не учитывает контекст и синонимы, не работает с “тонкими” различиями между senior/lead.**
- Это ожидаемо для baseline — сложные модели (трансформеры) должны показать прирост качества, особенно по малым и “тонким” классам.

##### Что делать дальше:
- Применить Roberta/MiniLM — они должны лучше различать смыслы и повысить recall по senior/lead.
- При необходимости — балансировать классы, использовать веса/аугментацию, дообучить модель на конкретных примерах.

**Базовая модель даёт рабочую отправную точку. Результат показывает, что текст вакансии частично отражает грейд, но есть большой потенциал для улучшения с помощью современных методов.**


Baseline-модель (TF-IDF + LogisticRegression) даёт accuracy 57%.
Лучше всего различает middle/junior, хуже — senior/lead.
Это ожидаемо для простой модели, и подтверждает необходимость использовать современные трансформеры для повышения качества.

#### Классификация грейда вакансии с помощью sentence-transformers MiniLM

Используем современную мультиязычную модель MiniLM для обучения классификатора уровня вакансии (junior/middle/senior/lead) по тексту.  
Такая модель учитывает смысл и контекст, что позволяет повысить качество предсказаний по сравнению с простыми ML-подходами.


In [13]:
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 850.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

#### Обучение и тестирование кастомного PyTorch-классификатора поверх sentence-transformers MiniLM

Модель получает sentence-эмбеддинги для текста вакансии и обучает поверх них простой линейный классификатор для определения грейда.  
Это "нейросетевой" способ, полностью на PyTorch, приближённый к профессиональным пайплайнам.

In [15]:
import torch
from sentence_transformers import SentenceTransformer, losses, InputExample
from torch.utils.data import DataLoader
import torch.nn as nn

# Загрузка базовой модели
base_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Freeze base (по желанию)
for param in base_model.parameters():
    param.requires_grad = False

# Добавляем классификационный слой
class MiniLMClassifier(nn.Module):
    def __init__(self, embed_dim, num_classes):
        super().__init__()
        self.base = base_model
        self.classifier = nn.Linear(embed_dim, num_classes)

    def forward(self, input_texts):
        with torch.no_grad():
            embeddings = self.base.encode(input_texts, convert_to_tensor=True)
        logits = self.classifier(embeddings)
        return logits

# Проверяем размерность эмбеддинга
embed_dim = base_model.get_sentence_embedding_dimension()
num_classes = 4
model = MiniLMClassifier(embed_dim, num_classes)


In [16]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

# Создаём кастомный датасет
class TextDataset(Dataset):
    def __init__(self, texts, labels, base_model):
        self.texts = texts
        self.labels = labels
        self.base_model = base_model

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        # Получаем sentence-эмбеддинг (без .detach(), потому что будем обучать classifier)
        with torch.no_grad():
            embedding = torch.tensor(self.base_model.encode(text, convert_to_numpy=True))
        return embedding, label

# train и test датасеты
train_dataset = TextDataset(X_train, y_train, base_model)
test_dataset = TextDataset(X_test, y_test, base_model)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MiniLMClassifier(embed_dim, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.001)
n_epochs = 5

for epoch in range(n_epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        embeddings, labels = batch
        embeddings, labels = embeddings.to(device), labels.to(device)
        outputs = model.classifier(embeddings)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * embeddings.size(0)
    avg_loss = total_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{n_epochs} - train loss: {avg_loss:.4f}")


Epoch 1/5 - train loss: 1.2506
Epoch 2/5 - train loss: 1.1851
Epoch 3/5 - train loss: 1.1536
Epoch 4/5 - train loss: 1.1320
Epoch 5/5 - train loss: 1.1132


In [18]:
from sklearn.metrics import classification_report, accuracy_score

model.eval()  # Переводим модель в режим оценки (отключает dropout/batchnorm)
all_preds = []
all_labels = []

# Отключаем градиенты для ускорения и экономии памяти
with torch.no_grad():
    for batch in test_loader:
        embeddings, labels = batch
        embeddings = embeddings.to(device)
        # Получаем логиты классификатора
        outputs = model.classifier(embeddings)
        # Выбираем класс с максимальным значением (argmax)
        preds = outputs.argmax(dim=1).cpu().numpy()
        # Собираем предсказания и реальные значения
        all_preds.extend(preds)
        all_labels.extend(labels.numpy())

# Считаем accuracy и полный классификационный отчёт (precision, recall, f1 по классам)
print("MiniLMClassifier test accuracy:", accuracy_score(all_labels, all_preds))
print("Классификационный отчёт:\n", classification_report(all_labels, all_preds, digits=3))

MiniLMClassifier test accuracy: 0.4691151919866444
Классификационный отчёт:
               precision    recall  f1-score   support

           0      0.446     0.239     0.311       155
           1      0.484     0.837     0.613       263
           2      0.393     0.174     0.241       138
           3      0.000     0.000     0.000        43

    accuracy                          0.469       599
   macro avg      0.331     0.312     0.291       599
weighted avg      0.418     0.469     0.405       599



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


ВЫВОДЫ позже

#### Дообучение Roberta на задаче классификации грейда

Используем ruRoberta (или Roberta-base), дообучаем на корпусе вакансий для задачи классификации грейда (junior/middle/senior/lead).  
Этот подход позволяет учесть глубокие семантические связи и контекст, что должно дать прирост качества по сравнению с простыми ML-моделями и MiniLM.
